# Projeto de Aprendizagem Automática II

## Procura de Exoplanetas no Espaço através da Emissão de Luz de Estrelas

### Importação de Bibliotecas

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

### Carregamento dos Dados

In [13]:
treino = pd.read_csv("../../../../Dados/dados_treino.csv")

In [14]:
teste = pd.read_csv("../../../../Dados/dados_teste.csv")

Em seguida é possível confirmar o carregamento dos dados a partir dos ficheiros.

In [15]:
treino.head()

,Unnamed: 0,LABEL,FLUX.0,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,...,FLUX.3187,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196
0,0,2,6.643575e-13,22.044950,346.424723,329.505904,1249.205479,182.654094,156.148092,103.977575,...,18.621552,159.049411,179.291372,103.977575,156.148092,182.654094,1249.205479,329.505904,346.424723,22.044950
1,1,2,5.324630e-13,315.058647,383.889750,399.050802,230.319773,346.075420,288.386248,123.347240,...,123.052404,147.877047,145.587727,123.347240,288.386248,346.075420,230.319773,399.050802,383.889750,315.058647
2,2,2,1.023182e-12,509.559899,148.713545,1298.642815,369.249675,672.540546,437.844536,295.045615,...,172.991190,354.523387,80.108584,295.045615,437.844536,672.540546,369.249675,1298.642815,148.713545,509.559899
3,3,2,7.638334e-14,119.852075,322.695474,668.866524,622.837768,1667.042377,479.142131,440.907070,...,165.281958,206.266502,339.671264,440.907070,479.142131,1667.042377,622.837768,668.866524,322.695474,119.852075
4,4,2,4.218847e-13,762.580219,344.996007,984.287978,997.920915,697.225208,783.809909,194.429858,...,33.054354,465.804574,244.741924,194.429858,783.809909,697.225208,997.920915,984.287978,344.996007,762.580219


O primeiro passo para testar os modelos é dividir os mesmos em variável de resposta e atributos.

In [22]:
X_train = treino.loc[:, treino.columns != 'LABEL'].values
y_train = treino.LABEL.values

X_test = teste.loc[:, teste.columns != 'LABEL'].values
y_test = teste.LABEL.values

### SVM

Em seguida deve ser feito um teste com os registos completos, ou seja, sem aplicar algum algoritmo de PCA. 

In [23]:
from sklearn.svm import SVC
clf = SVC(kernel="linear")
clf.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

Tendo o modelo treinado, é necessário prever os resultados do conjunto de treino. Assim, é possível observar se este modelo permite melhorias no que toca à classificação correta dos dados.

In [24]:
from sklearn import metrics
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_test, y_pred))
print("accuracy:", metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      0.99      0.99       565
           2       0.43      0.60      0.50         5

    accuracy                           0.99       570
   macro avg       0.71      0.80      0.75       570
weighted avg       0.99      0.99      0.99       570

accuracy: 0.9894736842105263


Como se pode observar pelas métricas em cima e pela matriz de confusão em seguida, a precisão na classificação da classe minoritária aumentou, tal como o *recall*, sendo este um algoritmo melhor do que o utilizado para testar o processamento de dados.

In [26]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[561   4]
 [  2   3]]


### PCA

No contexto de análise de séries temporais com *Machine Learning*, é extremamente importante a utilização de PCA, permitindo uma redução no número de atributos em estudo. Desta forma, selecionando-se um mínimo de 90% da explicação da variável de resposta por parte dos atributos, pode ser testado o mesmo modelo com menos dados.

In [27]:
from sklearn.decomposition import PCA
pca = PCA(0.90, True, False)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
print("the dimension of decomposed data is ", X_train.shape[1])

the dimension of decomposed data is  125


Tenha-se, assim, o modelo de SVM com os novos dados.

In [28]:
from sklearn.svm import SVC
clf = SVC(kernel="linear")
clf.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

Testando o conjunto de teste, é visível um aumento na precisão da classificação de sistemas com exoplanetas. Além disso, o valor de *recall* manteve-se igual.

In [29]:
from sklearn import metrics
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_test, y_pred))
print("accuracy:", metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       565
           2       0.60      0.60      0.60         5

    accuracy                           0.99       570
   macro avg       0.80      0.80      0.80       570
weighted avg       0.99      0.99      0.99       570

accuracy: 0.9929824561403509


Comparando a matriz de confusão agora obtida com a anterior, é visível que foram corretamente classificados 3 sistemas com exoplanetas em ambos os casos. Neste caso houve uma diminuição nas classificações erradas, sendo que 2 sistemas tinham sido classificados como tendo exoplanetas quando não era o caso. 

In [30]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[563   2]
 [  2   3]]


Assim, este algoritmo, aliado à análise de componentes principais mostrou possuir resultados bastante bons na classificações dos sistemas do problema, sendo que ainda existem aspetos que podem ser melhorados. Aspetos estes como a otimização de hiperparâmetros, numa tentativa de classificar corretamente os 5 sistemas com exoplanetas.